In [ ]:
import cv2
import numpy as np
import pandas as pd

from IPython.display import display
pd.set_option('display.max_colwidth', 0)

from nos.client import InferenceClient, TaskType

### Inference Client API

In [ ]:
client = InferenceClient()

In [ ]:
client.WaitForServer()

In [ ]:
client.IsHealthy()

### List all available models

In [ ]:
models = client.ListModels()
models_df = pd.DataFrame(models)
models_df

### Get model input/output signatures

In [ ]:
signatures = [client.GetModelInfo(spec).signature for spec in models]
pd.DataFrame(signatures)

### Load image

In [ ]:
import requests
from PIL import Image

url = "https://raw.githubusercontent.com/open-mmlab/mmdetection/main/demo/demo.jpg"
img = Image.open(requests.get(url, stream=True).raw).resize((640, 480))
img

## 1. Object detection

In [ ]:
images = [img]
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/nano", images=images)
for idx, (bboxes, scores, labels) in enumerate(zip(predictions["bboxes"], predictions["scores"], predictions["scores"])):
    print(idx, bboxes.shape, scores.shape, labels.shape)

### 1a. Visualize object detection

In [ ]:
def visualize_det2d(img: np.ndarray, bboxes: np.ndarray, labels: np.ndarray) -> np.ndarray:
    vis = np.asarray(img).copy()
    for bbox, label in zip(bboxes.astype(np.int32), labels):
        cv2.rectangle(vis, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    return vis

for idx, (img, bboxes, scores, labels) in enumerate(zip(images, predictions["bboxes"], predictions["scores"], predictions["scores"])):
    display(Image.fromarray(visualize_det2d(img, bboxes, labels)))

### 1b. Benchmark object detection

In [ ]:
%%timeit -n 10
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/nano", images=images)

## 2. Text-to-image generation with StableDiffusionV2

In [ ]:
prompts = ["fox jumped over the moon", "fox jumped over the sun"]
predictions = client.Run(TaskType.IMAGE_GENERATION, "stabilityai/stable-diffusion-2", 
                         prompts=prompts, width=512, height=512, num_images=1)

In [ ]:
for prompt, image in zip(prompts, predictions["images"]):
    print(prompt, image.size)
    display(image)

### Run image-embedding with CLIP

In [ ]:
predictions = client.Run(TaskType.IMAGE_EMBEDDING, "openai/clip", images=[img, img])
predictions["embedding"].shape

### Run text-embedding with CLIP

In [ ]:
predictions = client.Run(TaskType.TEXT_EMBEDDING, "openai/clip", texts=["fox jumped over the mooon", "fox jumped over the sun"])
predictions["embedding"].shape

### `client.Module` Interface

Get model handles for simpler remote-model execution. In this case, the following line re-uses the SDv2 model instantiated earlier.  

In [ ]:
sdv2 = client.Module(TaskType.IMAGE_GENERATION, "stabilityai/stable-diffusion-2")
predictions = sdv2(prompts=["fox in the rain at night, 4k, hdr"], width=512, height=512, num_images=1)
display(predictions["images"][0])

In [ ]:
det2d = client.Module(TaskType.OBJECT_DETECTION_2D, "yolox/small")
predictions = det2d(images=[img])

for idx, (img, bboxes, scores, labels) in enumerate(zip(images, predictions["bboxes"], predictions["scores"], predictions["scores"])):
    display(Image.fromarray(visualize_det2d(img, bboxes, labels)))